In [16]:
import ee
import geemap
import ipywidgets as widgets
from bqplot import pyplot as plt
from ipyleaflet import WidgetControl
from ipywidgets import interactive
import numpy as np

In [17]:
import os
#in local
work_dir = os.path.join(os.path.expanduser("~"), 'Documents')
texted_gif = os.path.join(work_dir,"voila1\\notebooks\landsat_ts_text.gif")

# Online app
#work_dir = os.path.join(os.path.expanduser("~"), 'notebooks')
#texted_gif = os.path.join(work_dir,"landsat_ts_text.gif")
#geemap.add_text_to_gif(out_gif, texted_gif, xy=('3%', '5%'), text_sequence=1985, font_size=30, font_color='#ffffff', add_progress_bar=False)

#label = 'Urban Growth in Beirut'
#geemap.add_text_to_gif(texted_gif, texted_gif, xy=('2%', '88%'), text_sequence=label, font_size=30, font_color='#ffffff', progress_bar_color='cyan')

geemap.show_image(texted_gif)

Output()

<h1>
<br>
 <br>
 </h1>
<center> <h1  style="background-color:#7CCA62; color:white;"> Sentinel-2 MSI: MultiSpectral Instrument, Level-2A</h1></center>
<h1>
<br>   
</h1>

In [18]:
Map1 = geemap.Map(center=(33.87, 35.9), zoom=5,add_google_map=False)
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')
Map1.addLayer(countries , {}, 'Country Boundaries');
Map1

Map(center=[33.87, 35.9], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(val…

In [ ]:

style = {'description_width': 'initial'}

output_widget11 = widgets.Output(layout={'border': '1px solid black'})
output_control11 = WidgetControl(widget=output_widget11, position='bottomright')
Map1.add_control(output_control11)

from_date = widgets.Text(
    description='From date:',
    value='2021-03-21',
    width=200,
    style=style
)

to_date = widgets.Text(
    description='To year:',
    value='2021-03-23',
    width=300,
    style=style
)

submit11 = widgets.Button(
    description='Submit',
    button_style='success',
    tooltip='Click me',
    style=style
)

band_1 = widgets.Dropdown(
    description='Band 1:',
    options=['','B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9',
              'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B',
              'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60'],
    value='B4',
    style=style
)

band_2 = widgets.Dropdown(
    description='Band 2:',
    options=['','B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9',
              'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B',
              'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60'],
    value='B3',
    style=style
)

band_3 = widgets.Dropdown(
    description='Band 3:',
    options=['','B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9',
              'B11', 'B12', 'AOT', 'WVP', 'SCL', 'TCI_R', 'TCI_G', 'TCI_B',
              'MSK_CLDPRB', 'MSK_SNWPRB', 'QA10', 'QA20', 'QA60'],
    value='B2',
    style=style
)

county_name = widgets.Text(
    description='Country name',
    value='Lebanon',
    width=350,
    style=style
)

full_widget11 = widgets.VBox([
    widgets.HBox([county_name]),
    widgets.HBox([from_date,to_date]),
    widgets.HBox([band_1, band_2,band_3]),
    submit11
])

full_widget11

In [ ]:
def maskS2clouds(image):
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0) \
    .And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    return image.updateMask(mask).divide(10000)


# Click event handler

def submit_clicked1(b):
    
    with output_widget11:
        output_widget11.clear_output()
        print('Computing...')
        Map1.default_style = {'cursor': 'wait'}

        try:
            start_date = from_date.value
            end_date = to_date.value
            
            county=county_name.value
            
            band1=band_1.value
            band2=band_2.value
            band3=band_3.value
            
           
            countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')
            country = countries.filter(ee.Filter.eq('ADM0_NAME', county))
            
            dataset = ee.ImageCollection('COPERNICUS/S2_SR') \
                  .filterDate(start_date, end_date) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)) \
                  .map(maskS2clouds)
            sentinel2 =dataset.mean()
            sentinel2 =sentinel2.clip(country)

            visualization = {
            'min': 0.0,
            'max': 0.3,
            'bands': [band1, band2, band3]
                }
            
            Map1.addLayer(sentinel2, visualization, f'{band1}/{band2}/{band3} _ {start_date}-{end_date}')
            
            output_widget11.clear_output()

            
            
            
    
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
        

        Map1.default_style = {'cursor': 'default'}

submit11.on_click(submit_clicked1)

In [ ]:
print('Change new layer opacity:')
land_layer = Map1.layers[-1]
land_layer.interact(opacity=(0, 1, 0.1))

<h1>
<br>
 <br>    
</h1>
<center> <h1  style="background-color:#7CCA62; color:white;"> Landsat timeseries</h1></center>
<h1>
<br>  
</h1>

In [ ]:
# center on Lebanon
lat = 33.87
lon = 35.9
Map = geemap.Map(center=[lat, lon], zoom=8, add_google_map=False)

# Add Earth Eninge layers to Map
countries = ee.FeatureCollection('FAO/GAUL/2015/level2').select('ADM0_NAME','ADM1_NAME','ADM2_NAME')
# Filter the feature collection to subset .
lebanon = countries.filter(ee.Filter.eq('ADM0_NAME', 'Lebanon'))
Map.addLayer(lebanon, {}, 'Second-level administrative region')


#Global/ALOS_landforms

#dataset_otq = ee.Image('CSP/ERGo/1_0/Global/ALOS_landforms')
#dataset_otq =dataset_otq.clip(lebanon)
#Map.addLayer(dataset_otq, {}, "CSP/ERGo/1_0/Global/ALOS_landforms")

'''
#Add Earth Engine dataset

dem = ee.Image('USGS/SRTMGL1_003')
dem=dem.clip(lebanon)
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}
Map.addLayer(dem, vis_params, 'STRM DEM', True, 1)
'''

'''
#landsat7 = ee.Image('LE7_TOA_5YEAR/1999_2003')
#landsat7=landsat7.clip(lebanon)
#Map.addLayer(landsat7, {'bands': ['B4', 'B3', 'B2'], 'min': 20, 'max': 200}, 'Landsat 7')
'''

'''
#Global/ALOS_landforms
#landcover = ee.Image("ESA/GLOBCOVER_L4_200901_200912_V2_3").select('landcover')
#landcover=landcover.clip(lebanon)
#Map.addLayer(landcover, {}, 'Land cover')
'''




'''
#legend_keys = ['black','white','orange','yellow','red']
#colorS can be defined using either hex code or RGB (0-255, 0-255, 0-255)
legend_colors = [(0, 0,0), (255,255, 255), (251, 125, 0),(251, 223, 0), (255, 0 ,0)]

for i in range(2020, 2021, 1):
    for j in range(1, 2, 1):
        if (j== 2):
        
            d=28
        
        elif (j==4 or j==6 or j==9 or j==11  ): 
            d=30
        
        else: 
            d=31
        

        dataset = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG')\
        .filterDate(f'{i}-{j}-01', f'{i}-{j}-{d}')
        nighttime =dataset.sort('system:time_end').first().select('avg_rad')
        nighttime = nighttime.clip(lebanon)
        nighttimeVis = {'min': 0.0, 'max': 60.0, 'palette':['black','white','orange','yellow','red']}
    #Map.setCenter(35.9957, 33.8718, 8)
    #Map.addLayer(nighttime, nighttimeVis, 'Nighttime '+i+'-'+j,False)
#print(f'{i}-{j}-{d}')
        Map.addLayer(nighttime , nighttimeVis, f'night light {i}-{j}-{d}')
        
        '''



#Map.add_legend(legend_title="night",legend_keys=legend_keys, legend_colors=legend_colors, position='bottomleft')
Map

In [ ]:
# Designe interactive widgets

style = {'description_width': 'initial'}

output_widget = widgets.Output(layout={'border': '1px solid black'})
output_control = WidgetControl(widget=output_widget, position='bottomright')
Map.add_control(output_control)

admin1_widget = widgets.Text(
    description='First-level administrative region:',
    value='Bekaa',
    width=200,
    style=style
)

admin2_widget = widgets.Text(
    description='Second-level administrative area:',
    value='Rachiaya',
    width=300,
    style=style
)

band_combo = widgets.Dropdown(
    description='Display image band combination:',
    options=['Red/Green/Blue', 'NIR/Red/Green',  'SWIR2/SWIR1/NIR', 'NIR/SWIR1/Red','SWIR2/NIR/Red', 
             'SWIR2/SWIR1/Red', 'SWIR1/NIR/Blue', 'NIR/SWIR1/Blue', 'SWIR2/NIR/Green', 'SWIR1/NIR/Red'],
    value='Red/Green/Blue',
    style=style
)

year_widget = widgets.IntSlider(min=2010, max=2020, value=2020, description='Display data for the specified year:', width=400, style=style)

fmask_widget = widgets.Checkbox(
    value=True,
    description='Apply mask (remove clouds, shadows, snow)',
    style=style
)


# Normalized Satellite Indices: https://www.usna.edu/Users/oceano/pguth/md_help/html/norm_sat.htm

nd_options = ['Vegetation Index (NDVI)', 
              'Water Index (NDWI)',
              'Modified Water Index (MNDWI)',
              'Snow Index (NDSI)',
              'Soil Index (NDSI)',
              'Burn Ratio (NBR)',
              'Customized']
nd_indices = widgets.Dropdown(options=nd_options, value='Water Index (NDWI)', description='Normalized index:', style=style)

first_band = widgets.Dropdown(
    description='Band 1:',
    options=['Blue', 'Green','Red','NIR', 'SWIR1', 'SWIR2'],
    value='Green',
    style=style
)

second_band = widgets.Dropdown(
    description='Band 2:',
    options=['Blue', 'Green','Red','NIR', 'SWIR1', 'SWIR2'],
    value='SWIR1',
    style=style
)

nd_threshold = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.01,
    description='Threshold:',
    orientation='horizontal',
    style=style
)

nd_color = widgets.ColorPicker(
    concise=False,
    description='colour:',
    value='blue',
    style=style
)

def nd_index_change(change):
    if nd_indices.value == 'Vegetation Index (NDVI)':
        first_band.value = 'NIR'
        second_band.value = 'Red'
    elif nd_indices.value == 'Water Index (NDWI)':
        first_band.value = 'NIR'
        second_band.value = 'SWIR1'   
    elif nd_indices.value == 'Modified Water Index (MNDWI)':
        first_band.value = 'Green'
        second_band.value = 'SWIR1'   
    elif nd_indices.value == 'Snow Index (NDSI)':
        first_band.value = 'Green'
        second_band.value = 'SWIR1'
    elif nd_indices.value == 'Soil Index (NDSI)':
        first_band.value = 'SWIR1'
        second_band.value = 'NIR'        
    elif nd_indices.value == 'Burn Ratio (NBR)':
        first_band.value = 'NIR'
        second_band.value = 'SWIR2'
    elif nd_indices.value == 'Customized':
        first_band.value = None
        second_band.value = None
        
nd_indices.observe(nd_index_change, names='value')

submit = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

full_widget = widgets.VBox([
    widgets.HBox([admin1_widget, admin2_widget]),
    widgets.HBox([band_combo, year_widget, fmask_widget]),
    widgets.HBox([nd_indices, first_band, second_band, nd_threshold, nd_color]),
    submit
])

full_widget

In [ ]:
# Capture user interaction with the map

def handle_interaction(**kwargs):
    latlon = kwargs.get('coordinates')
    if kwargs.get('type') == 'click':
        Map.default_style = {'cursor': 'wait'}
        xy = ee.Geometry.Point(latlon[::-1])
        selected_fc = lebanon.filterBounds(xy)
        
        with output_widget:
            output_widget.clear_output()
        
            try:
                admin1_id = selected_fc.first().get('ADM1_NAME').getInfo()
                admin2_id = selected_fc.first().get('ADM2_NAME').getInfo()
                admin1_widget.value = admin1_id
                admin2_widget.value = admin2_id
                Map.layers = Map.layers[:4]        
                geom = selected_fc.geometry()
                layer_name = admin1_id + '-' + admin2_id
                Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)  
                print(layer_name)
            except:
                print('Cannot find the relevant administrative district')
                Map.layers = Map.layers[:4]
            
        Map.default_style = {'cursor': 'pointer'}

Map.on_interaction(handle_interaction)

In [ ]:
# Click event handler

def submit_clicked(b):
    
    with output_widget:
        output_widget.clear_output()
        print('Computing...')
        Map.default_style = {'cursor': 'wait'}

        try:
            admin1_id = admin1_widget.value
            admin2_id = admin2_widget.value
            band1 = first_band.value
            band2 = second_band.value
            selected_year = year_widget.value
            threshold = nd_threshold.value
            bands = band_combo.value.split('/')
            apply_fmask = fmask_widget.value
            palette = nd_color.value
            
            roi = lebanon.filter(ee.Filter.And(ee.Filter.eq('ADM1_NAME', admin1_id), ee.Filter.eq('ADM2_NAME', admin2_id)))
            Map.layers = Map.layers[:4]        
            geom = roi.geometry()
            layer_name = admin1_id + '-' + admin2_id
            Map.addLayer(ee.Image().paint(geom, 0, 2), {'palette': 'red'}, layer_name)  
            
            images = geemap.landsat_timeseries(roi=roi, start_year=2005, end_year=2020, start_date='01-01', end_date='12-31', apply_fmask=apply_fmask)
            nd_images = images.map(lambda img: img.normalizedDifference([band1, band2]))
            result_images = nd_images.map(lambda img: img.gt(threshold))

            selected_image = ee.Image(images.toList(images.size()).get(selected_year - 2010))
            selected_result_image = ee.Image(result_images.toList(result_images.size()).get(selected_year - 2010)).selfMask()
            
            vis_params = {
                'bands': bands,
                'min': 0,
                'max': 3000
            }
            
            Map.addLayer(selected_image, vis_params, 'Landsat ' + str(selected_year))
            Map.addLayer(selected_result_image, {'palette': palette}, 'Result ' + str(selected_year))

            
            def cal_area(img):
                pixel_area = img.multiply(ee.Image.pixelArea()).divide(1e6)
                img_area = pixel_area.reduceRegion(**{
                    'geometry': roi.geometry(),
                    'reducer': ee.Reducer.sum(),
                    'scale': 1000,
                    'maxPixels': 1e12,
                    'bestEffort': True
                })
                return img.set({'area': img_area})
            
            areas = result_images.map(cal_area)
            stats = areas.aggregate_array('area').getInfo()
            x = list(range(2010, 2021))
            y = [item.get('nd') for item in stats]
            
            fig = plt.figure(1)
            fig.layout.height = '270px'
            plt.clear()
            plt.plot(x, y)
            plt.title('Temporal trend (2005-2021)')
            plt.xlabel('Year')
            plt.ylabel('Area (km2)')
            
            output_widget.clear_output()            

            plt.show()
    
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
        

        Map.default_style = {'cursor': 'default'}

submit.on_click(submit_clicked)

In [ ]:
print('Change Land cover layer opacity:')
land_layer = Map.layers[-2]
land_layer.interact(opacity=(0, 1, 0.1))

<h1>
  <br>
   <br>
</h1>    
<center> <h1  style="background-color:#7CCA62; color:white;"> Visualizing weather data</h1></center>
<h1>
  <br> 
</h1>  

In [12]:
# center on Lebanon
# Add Earth Eninge layers to 

Map2 = geemap.Map(center=[35.8926, 33.7835], zoom=4, add_google_map=False)
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')
# Filter the feature collection to subset .
lebanon = countries.filter(ee.Filter.eq('ADM0_NAME', 'Lebanon'))


Map2

Map(center=[35.8926, 33.7835], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButto…

In [13]:
style = {'description_width': 'initial'}

output_widget2 = widgets.Output(layout={'border': '1px solid black'})
output_control2 = WidgetControl(widget=output_widget2, position='bottomright')
Map2.add_control(output_control2)

admin1_widget = widgets.Text(
    description='From date:',
    value='2021-03-21',
    width=200,
    style=style
)

admin2_widget = widgets.Text(
    description='To date:',
    value='2021-03-22',
    width=300,
    style=style
)
submit2 = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click me',
    style=style
)

full_widget2 = widgets.VBox([
    widgets.HBox([admin1_widget, admin2_widget]),
    submit2
])

full_widget2

In [14]:
# Click event handler

def submit_clicked2(b):
    
    with output_widget2:
        output_widget2.clear_output()
        print('Computing...')
        Map2.default_style = {'cursor': 'wait'}

        try:
            admin1_id = admin1_widget.value
            admin2_id = admin2_widget.value
           
            
            
            layer_name = admin1_id + '-' + admin2_id
           
            
            collection = ee.ImageCollection('NOAA/GFS0P25') \
            .filterDate(admin1_id , admin2_id ) \
            .limit(24) \
            .select('temperature_2m_above_ground')

            vis_params = {
                'min': -40.0,
                'max': 35.0,
                'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
            }
            
            
            legend_keys=['-40 : -30','-31 : -20','-21 : -6','-5 : 9','10 : 22', '23 : 35']
            legend_colors = [(0,0,255), (128,0,128), (0,255,255),(0,128,0), (255,255,0),(255,0,0)]
            #legend_colors=['blue', 'purple', 'cyan', 'green', 'yellow', 'red']
            first_image = collection.first()
            #first_image=first_image.clip(lebanon)
            Map2.addLayer(first_image, vis_params, "First image")
            Map2.add_legend(legend_title="Temperature Above Ground",legend_keys=legend_keys,legend_colors=legend_colors, position='bottomleft')
            
            
            image = collection.toBands()
            Map2.addLayer(image, {}, "Time series", False)
            labels = [str(n).zfill(2) + ":00" for n in range(0, 24)]
            Map2.add_time_slider(collection, vis_params, labels=labels, time_interval=1)
            Map2.addLayer(lebanon, {}, 'Second-level administrative region')
           
            output_widget2.clear_output()
            
            
            
            
    
        except Exception as e:
            print(e)
            print('An error occurred during computation.')
        

        Map2.default_style = {'cursor': 'default'}

submit2.on_click(submit_clicked2)

In [15]:
print('Lebanon :')
land_layer = Map2.layers[-1]
land_layer.interact(opacity=(0, 1, 1))

Lebanon :


Box(children=(IntSlider(value=1, description='opacity', max=1),))

<h1>
<br>
<br>
<br>
<br>
<br>   
</h1>  

<h3 align="center"> © Hisham Sharrouf, Data Science student at LU. </h3>